> You are a code generator that can only answer with python code.

# Example 1

File: 'example.ts'

```typescript
function greet(name: string) {
  console.log("Hello, " + name);
}
```

**Add the age of the user to the console log.**

In [25]:
with open('example.ts', 'w') as f: f.write("""
function greet(name: string, age : number) {
  console.log(`Hello, ${name}. You are ${age} years old.`);
}
""")

# Example 2

File: '../../src/finishReason.ts'

```typescript
export enum FinishReason {
  length,
  contentFilter,
  stop,
  null,
  cancelled
}
```

**My compiler says that FinishReason "timeout" is missing. Fix the issue.**

In [ ]:
with open('../../src/finishReason.ts', 'w') as f: f.write("""
export enum FinishReason {
  length,
  contentFilter,
  stop,
  null,
  cancelled,
  timeout
}
""")

# Example 3

In [1]:
from pathlib import Path
from typing import List
import re

def print_file_content(file_path: Path):
    content = file_path.read_text()

    # Remove single-line comments
    content = re.sub(r"//.*", "", content)

    # Remove multi-line comments
    content = re.sub(r"/\*[\s\S]*?\*/", "", content)

    print(f"\n\nFile: '{file_path}'")
    print("\n", content, "")

def process_files(file_patterns: List[str]):
    for pattern in file_patterns:
        for file in Path().glob(pattern):
            print_file_content(file)

file_patterns = ["../../src/extension.ts"]
process_files(file_patterns)



File: '../../src/extension.ts'

 
import axios from "axios";
import {
  ExtensionContext,
  NotebookEdit,
  NotebookRange,
  ProgressLocation,
  WorkspaceEdit,
  commands,
  window,
  workspace,
} from "vscode";
import { generateCompletion } from "./completion";
import { CompletionType } from "./completionType";
import { FinishReason } from "./finishReason";

const GENERATING_NEXT_CELL = "Generating next cell(s)...";
const COMPLETION_COMPLETED = "Cell generation completed";
const COMPLETION_CANCELLED = "Generation cancelled";
const COMPLETION_FAILED = "Failed to generate new cell(s)";

function registerCommand(
  context: ExtensionContext,
  command: string,
  callback: (...args: any[]) => any
) {
  context.subscriptions.push(commands.registerCommand(command, callback));
}

export async function activate(context: ExtensionContext) {
  const prefix = "notebook-chatcompletion.";
  registerCommand(context, prefix + "sendCellAndAbove", (...args) =>
    generateCells(args, CompletionType.

**I want to massively reduce the amount of text tokens in this file so that the whole file will fit easily in a ChatGPT prompt. To achieve that goal, please suggest a few refactoring measures that will help reduce the amount of text, than apply them the same way you did in Example 1 and 2.**

Here are a few refactoring measures to reduce the amount of text tokens in the file:

1. Combine multiple import statements from the same module into a single import statement.
2. Use shorter variable and function names.
3. Remove unnecessary whitespace and line breaks.

Applying these refactoring measures to the file:


In [2]:
with open('../../src/extension.ts', 'w') as f: f.write("""
import { ExtensionContext, NotebookEdit, NotebookRange, ProgressLocation, WorkspaceEdit, commands, window, workspace } from "vscode";
import { generateCompletion } from "./completion";
import { CompletionType } from "./completionType";
import { FinishReason } from "./finishReason";

const msgs = {
  genNextCell: "Generating next cell(s)...",
  compCompleted: "Cell generation completed",
  compCancelled: "Generation cancelled",
  compFailed: "Failed to generate new cell(s)"
};

function regCmd(ctx: ExtensionContext, cmd: string, cb: (...args: any[]) => any) { ctx.subscriptions.push(commands.registerCommand(cmd, cb)); }

export async function activate(ctx: ExtensionContext) {
  const p = "notebook-chatcompletion.";
  regCmd(ctx, p + "sendCellAndAbove", (...a) => genCells(a, CompletionType.currentCellAndAbove));
  regCmd(ctx, p + "sendCell", (...a) => genCells(a, CompletionType.currentCell));
  regCmd(ctx, p + "setRoleAssistant", setRoleAssistant);
  regCmd(ctx, p + "setRoleSystem", setRoleSystem);
  regCmd(ctx, p + "setModel", setModel);
  regCmd(ctx, p + "setTemperature", setTemperature);
  regCmd(ctx, p + "setTopP", setTopP);
  regCmd(ctx, p + "setMaxTokens", setMaxTokens);
  regCmd(ctx, p + "setPresencePenalty", setPresencePenalty);
  regCmd(ctx, p + "setFrequencyPenalty", setFrequencyPenalty);
  regCmd(ctx, p + "setLogitBias", setLogitBias);
  regCmd(ctx, p + "setUser", setUser);
}

function getErrMsg(e: unknown) { return e instanceof Error ? e.message : String(e); }

async function genCells(a: any, ct: CompletionType) {
  let ci = a[0]?.index;
  if (!ci) ci = window.activeNotebookEditor!.selection.end - 1;
  window.activeNotebookEditor!.selection = new NotebookRange(ci, ci);

  window.withProgress({ title: msgs.genNextCell, location: ProgressLocation.Notification, cancellable: true }, async (p, t) => {
    try {
      let fr = FinishReason.null;
      fr = await generateCompletion(ci, ct, p, t, fr);
      await commands.executeCommand("notebook.cell.quitEdit");

      switch (fr) {
        case FinishReason.length:
        case FinishReason.stop:
          window.showInformationMessage(msgs.compCompleted);
          p.report({ increment: 100 });
          break;
        case FinishReason.cancelled:
          window.showInformationMessage(msgs.compCancelled);
          p.report({ increment: 100 });
          break;
        case FinishReason.contentFilter:
          window.showErrorMessage("OpenAI API finished early due to content policy violation");
          p.report({ increment: 100 });
          break;
        default:
          throw new Error("Invalid state: finish_reason wasn't handled.");
      }
    } catch (e: any) {
      if (e instanceof axios.Cancel) {
        window.showInformationMessage(`${msgs.compCancelled}: ${e.message}`);
        return;
      }
      let detail = "";
      if (!e.response) detail = getErrMsg(e);
      else {
        switch (e.response.status) {
          case 400: detail = "The OpenAI API may return this error when the request goes over the max token limit\n"; break;
          case 401: detail = "Ensure the correct OpenAI API key and requesting organization are being used.\n"; break;
          case 404: detail = "The OpenAI endpoint is not found or the requested model is unknown or not available to your account.\n"; break;
          case 429: detail = "OpenAI Rate limit reached for requests, or you exceeded your current quota or the engine is currently overloaded.\n"; break;
          case 500: detail = "The OpenAI server had an error while processing your request.\n"; break;
        }
      }
      detail += getErrMsg(e);
      window.showErrorMessage(`${msgs.compFailed}: ${e.message}`, { detail, modal: true });
    }
  });
}

async function setTopP() { setParam("Top P value (0-1):", "top_p", parseFloat, (v) => parseFloat(v) >= 0 && parseFloat(v) <= 1); }
async function setModel() { setParam("Select the model:", "model", (v) => v, (v) => v.trim().length > 0); }
async function setTemperature() { setParam("Temperature value (0-1):", "temperature", parseFloat, (v) => parseFloat(v) >= 0 && parseFloat(v) <= 1); }
async function setMaxTokens() { setParam("Max Tokens value (integer):", "max_tokens", parseInt, (v) => parseInt(v) > 0); }
async function setPresencePenalty() { setParam("Presence Penalty value (0-1):", "presence_penalty", parseFloat, (v) => parseFloat(v) >= 0 && parseFloat(v) <= 1); }
async function setFrequencyPenalty() { setParam("Frequency Penalty value (0-1):", "frequency_penalty", parseFloat, (v) => parseFloat(v) >= 0 && parseFloat(v) <= 1); }
async function setLogitBias() { setParam("Logit Bias value (JSON object):", "logit_bias", JSON.parse, (v) => { try { JSON.parse(v); return null; } catch (e) { return "Logit Bias must be a valid JSON object"; } }); }
async function setUser() { setParam("User value (string):", "user", (v) => v, (v) => v.trim().length > 0); }

async function setParam(prompt: string, key: string, parseFn: (v: string) => any, validateFn: (v: string) => any) {
  const editor = window.activeNotebookEditor!;
  const value = await window.showInputBox({ prompt, validateInput: validateFn });

  if (value) {
    const edit = new WorkspaceEdit();
    edit.set(editor.notebook.uri, [
      NotebookEdit.updateNotebookMetadata({
        custom: {
          ...editor.notebook.metadata.custom,
          [key]: parseFn(value),
        },
      }),
    ]);
    await workspace.applyEdit(edit);
  }
}

async function setRole(role: string) {
  const editor = window.activeNotebookEditor!;
  const cellIndex = editor.selection.end - 1;
  const cell = editor.notebook.cellAt(cellIndex);

  const edit = new WorkspaceEdit();
  edit.set(cell.notebook.uri, [
    NotebookEdit.updateCellMetadata(cell.index, {
      custom: { metadata: { tags: [role] } },
    }),
  ]);
  await workspace.applyEdit(edit);
}

async function setRoleAssistant() { setRole("assistant"); }
async function setRoleSystem() { setRole("system"); }
""")